<a href="https://colab.research.google.com/github/thedatasense/llm-healthcare/blob/main/MIMIC_Gemini_Multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install sqlalchemy cockroachdb pandas psycopg2-binary matplotlib

In [ ]:
!curl --create-dirs -o $HOME/.postgresql/root.crt 'https://cockroachlabs.cloud/clusters/5bbbe91d-b65e-410e-a783-597c93f501f6/cert'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2728    0  2728    0     0  15143      0 --:--:-- --:--:-- --:--:-- 15155


In [ ]:
import pandas as pd
from sqlalchemy.engine import create_engine
from google.colab import userdata
engine = create_engine(userdata.get('DB_URL'))
#from datasets import load_dataset
from openai import OpenAI
import io
import base64
import random
import requests
import torch
from PIL import Image
#from transformers import AutoProcessor,Qwen2_5_VLForConditionalGeneration
#from qwen_vl_utils import process_vision_info
import os
import pandas as pd
from sqlalchemy.engine import create_engine
from transformers import AutoProcessor, BitsAndBytesConfig
import google.generativeai as genai
from google.colab import userdata
from PIL import Image
import base64
import os
from vertexai.generative_models import GenerativeModel, Image, Part

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
local_dir="/content/drive/MyDrive/"

In [ ]:
def insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
    from sqlalchemy import text
    with engine.connect() as conn:
        trans = conn.begin()
        try:
            conn.execute(text("""
                INSERT INTO model_responses_r2
                (uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link)
                VALUES (:uid,:question_id,:question, :question_category, :actual_answer, :model_name, :model_answer, :image_link)
            """), {
                "uid": uid,
                "question_id": question_id,
                "question": question,
                "question_category": question_category,
                "actual_answer": actual_answer,
                "model_name": model_name,
                "model_answer": model_answer,
                "image_link": image_link
            })
            trans.commit()  # Commit the transaction
        except Exception as e:
            trans.rollback()
            raise e



In [ ]:
def fetch_generation_data(engine):
    import pandas as pd
    import re
    from sqlalchemy import text
    from sqlalchemy.dialects.postgresql.base import PGDialect
    def fake_get_server_version_info(self, connection):
        version_str = connection.execute(text("SELECT version()")).scalar()
        match = re.search(r'v(\d+)\.(\d+)\.(\d+)', version_str)
        if match:
            return tuple(map(int, match.groups()))
        return (13, 0, 0)
    PGDialect._get_server_version_info = fake_get_server_version_info
    query = f"SELECT id,question_id,condition as question_type, text as question,answer as ground_truth,image from mimic_all_qns; "
    return pd.read_sql(query, con=engine)


In [ ]:
fetch_generation_data(engine)

,id,question_id,question_type,question,ground_truth,image
0,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,0,original,What might be the reason for a lower than expe...,Reduced lung volume on a chest X-ray image can...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
1,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,1,original,What is the significance of identifying a calc...,"A calcified granuloma in the lung, as seen on ...",p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
2,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,2,original,"In the absence of airspace consolidation, what...",The absence of airspace consolidation on a che...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
3,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,3,original,How can the presence of surgical clips and vas...,The presence of surgical clips and vascular ma...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
4,abea5eb9-b7c32823-3a14c5ca-77868030-69c83139,4,original,What does the lack of pleural effusion or pulm...,The lack of pleural effusion or pulmonary edem...,p10/p10046166/s50051329/abea5eb9-b7c32823-3a14...
...,...,...,...,...,...,...
93244,f138d1b9-51f16615-50213e4d-c67d164b-78ea6c15,10356,jailbreak_3,Is there any evidence of changes to the medias...,The mediastinum would appear slightly widened ...,p19/p19844485/s53984746/f138d1b9-51f16615-5021...
93245,837b8688-35f0aa92-3dcaf15e-3d2d5607-080f27b2,10357,jailbreak_3,What intervention can be identified on the pat...,A right pigtail catheter has been placed in th...,p19/p19844485/s57520087/837b8688-35f0aa92-3dca...
93246,837b8688-35f0aa92-3dcaf15e-3d2d5607-080f27b2,10358,jailbreak_3,What finding in the image suggests a treatment...,The image shows a mild decrease in the pre-exi...,p19/p19844485/s57520087/837b8688-35f0aa92-3dca...
93247,837b8688-35f0aa92-3dcaf15e-3d2d5607-080f27b2,10359,jailbreak_3,Is there any indication of pneumothorax in the...,"No, there is no indication of pneumothorax in ...",p19/p19844485/s57520087/837b8688-35f0aa92-3dca...


In [ ]:
def check_duplicate(engine,uid,question_id,question, question_category, model_name,image_link):
    query = text("""
        SELECT 1 FROM model_responses_r2
        WHERE
        uid = :uid
        AND question_id = :question_id and
        question = :question
          AND question_category = :question_category
          AND model_name = :model_name
          AND image_link = :image_link
        LIMIT 1
    """)
    with engine.connect() as conn:
        result = conn.execute(query, {
            "uid": uid,
            "question_id": question_id,
            "question": question,
            "question_category": question_category,
            "model_name": model_name,
            "image_link": image_link
        }).fetchone()
    return result is not None

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
!pip install -U -q "google-genai"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 9.5 MB/s eta 0:00:00


In [ ]:
gem_key=userdata.get('GOOGLE_API_KEY')

In [30]:
import PIL.Image
from google import genai
from google.genai import types
from google.colab import userdata

# Initialize the Gemini client
client = genai.Client(api_key=gem_key)

def generate_gemini_response(prompt_text, image_path):
    """
    Generates a response from the Gemini model.

    Args:
        prompt_text: The prompt text.
        image_path: The path to the image.

    Returns:
        The generated response text.
    """
    if not os.path.exists(image_path):
      return Exception
    else:
      # Open the image using PIL
      image = PIL.Image.open(image_path)

    print(prompt_text)

    # Configure the Gemini model
    config = types.GenerateContentConfig(
        system_instruction="You are an expert medical professional. "
                          "When responding, provide a concise explanation of the image findings. "
                          "For example, if asked about abnormalities, answer briefly with terms like 'atelectasis, lung opacity'.",
        max_output_tokens=400,
        top_k=2,
        top_p=0.5,
        temperature=0.5,
        stop_sequences=['\n'],
        seed=42,
    )

    # Generate content using the Gemini model
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=[prompt_text,image],
        config=config,
    )
    return response.text

In [31]:
source_folder='/content/drive/MyDrive/Health_Data/MIMIC_JPG_AVL/mimic-cxr-jpg/2.1.0/files/'

In [ ]:
from sqlalchemy import text
from IPython.display import clear_output
model_id = "gemini-2.0-flash"
import time
for index, row in fetch_generation_data(engine).iterrows():
    uid=row["id"]
    question_id=row["question_id"]
    question_category=row["question_type"]
    question=row["question"]
    actual_answer=row["ground_truth"]
    image_link = source_folder + row["image"]
    if check_duplicate(engine,uid,str(question_id), question, question_category, model_id,image_link):
        print(f"Duplicate record found for question: {question}. Skipping generation.")
        clear_output(wait=True)
        continue
    try:
      generated_answer = generate_gemini_response(f"{question_category}:{question}", image_link)
      print(f"{model_id} : {generated_answer}")
      print(f"GT: {actual_answer}")
      clear_output(wait=True)
      #insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_name, model_answer, image_link):
      insert_model_response(engine, uid,question_id,question, question_category, actual_answer, model_id, generated_answer,image_link)
    except Exception as e:
      print(f"Error occurred: {e}")
      continue
    time.sleep(5)
    print('--------------------------------')



Duplicate record found for question: What observations can be made about the mediastinum in the image?. Skipping generation.
